# Análisis y Modelado de Series de Tiempo de Combustibles en Guatemala: Consumo, Importación y Precios (2000–2025)

## Carga, limpieza y preparación de datos

### Librerias

In [114]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA

# import pmdarima as pm  # auto_arima

from prophet import Prophet

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neural_network import MLPRegressor  # NN simple o usar tensorflow dependerá por ahora el simple

### Variables - Constantes

In [ ]:
DATA_DIR = "./data-input"
OUTPUT_DIR = "./data-output"

# input files
CONSUMO_2024_XLSX = f"{DATA_DIR}/CONSUMO-HIDROCARBUROS-2024-12.xlsx"
CONSUMO_2025_XLSX = f"{DATA_DIR}/VENTAS-HIDROCARBUROS-2025-05.xlsx"
IMPORT_2024_XLSX = f"{DATA_DIR}/IMPORTACION-HIDROCARBUROS-VOLUMEN-2024-12.xlsx"
IMPORT_2025_XLSX = f"{DATA_DIR}/IMPORTACION-HIDROCARBUROS-VOLUMEN-2025-05.xlsx"
PRECIOS_2025_XLSX = f"{DATA_DIR}/Precios-Promedio-Nacionales-Diarios-2025-1.xlsx"

# output files
CONSUMO_CSV = f"{OUTPUT_DIR}/consumo_combustibles.csv"
IMPORT_CSV = f"{OUTPUT_DIR}/importacion_combustibles.csv"
PRECIOS_CSV = f"{OUTPUT_DIR}/precios_diarios.csv"

# var control
SAVE_CONSUMO_CSV = True
SAVE_IMPORT_CSV = True
SAVE_PRECIOS_CSV = True
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [165]:
df_consumo_raw_2024 = pd.read_excel(CONSUMO_2024_XLSX, skiprows=6)
df_consumo_raw_2024.head()
print(f"Consumo 2024: {df_consumo_raw_2024.shape[0]} rows, {df_consumo_raw_2024.shape[1]} columns")

Consumo 2024: 303 rows, 24 columns


In [166]:
df_consumo_raw_2024.tail()

,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gas natural,Gasolina de aviación,Gasolina regular,Gasolina superior,Grasas lubricantes,Kerosina,Mezclas\noleosas,Naftas,PetCoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,Orimulsión,Total
298,2024-11-01 00:00:00,27908.70,40511.20,124919.31,443.86,82838.52,1307671.59,25612.05,597059.97,0.00,1314.62,826513.65,699028.58,-2155.73,-404.93,2222.71,213.66,458140.38,77270.48,12412.73,0.0,0.0,0.0,4281521.35
299,2024-12-01 00:00:00,32294.67,3193.68,138019.95,-105.04,52651.66,1474651.06,24615.04,564762.33,-708.53,45840.30,913940.78,773715.97,247.81,-1351.58,1228.81,107.08,320785.16,90160.00,14279.61,0.0,0.0,0.0,4448328.76
300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,Fuente: informes mensuales de titulares de lic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,Nota: Información sujeta a actualización por r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
df_consumo_raw_2024 = df_consumo_raw_2024.iloc[:-3]

In [168]:
df_consumo_raw_2024.tail()

,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gas natural,Gasolina de aviación,Gasolina regular,Gasolina superior,Grasas lubricantes,Kerosina,Mezclas\noleosas,Naftas,PetCoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,Orimulsión,Total
295,2024-08-01 00:00:00,30900.55,17003.78,156542.43,-60.64,89476.43,1218077.03,14935.62,551959.94,-4951.56,1503.47,869998.00,707708.07,-2014.59,75.78,1134.75,108.90,865931.97,84348.69,10368.44,0.0,0.0,0.0,4613047.06
296,2024-09-01 00:00:00,14622.84,14328.19,129425.22,-270.15,78778.74,1162290.44,14095.76,555683.08,-3573.61,574.16,827629.00,685834.74,1082.53,118.16,832.23,92.59,491181.54,78185.20,17725.02,0.0,0.0,0.0,4068635.68
297,2024-10-01 00:00:00,107.02,14598.41,165763.38,452.66,78247.37,1398782.97,15216.16,600453.87,0.00,388.41,942394.34,790948.04,-1692.10,82.29,1095.07,301.66,487452.36,88885.31,15835.09,0.0,0.0,0.0,4599312.31
298,2024-11-01 00:00:00,27908.70,40511.20,124919.31,443.86,82838.52,1307671.59,25612.05,597059.97,0.00,1314.62,826513.65,699028.58,-2155.73,-404.93,2222.71,213.66,458140.38,77270.48,12412.73,0.0,0.0,0.0,4281521.35
299,2024-12-01 00:00:00,32294.67,3193.68,138019.95,-105.04,52651.66,1474651.06,24615.04,564762.33,-708.53,45840.30,913940.78,773715.97,247.81,-1351.58,1228.81,107.08,320785.16,90160.00,14279.61,0.0,0.0,0.0,4448328.76


In [170]:
print("Eliminación de filas innecesarias completada.")
print(f"Consumo 2024: {df_consumo_raw_2024.shape[0]} rows, {df_consumo_raw_2024.shape[1]} columns")
print(df_consumo_raw_2024.columns)

Eliminación de filas innecesarias completada.
Consumo 2024: 300 rows, 24 columns
Index(['Fecha', 'Aceites lubricantes', 'Asfalto', 'Bunker', 'Ceras',
       'Combustible turbo jet', 'Diesel bajo azufre',
       'Diesel ultra bajo azufre', 'Gas licuado de petróleo', 'Gas natural',
       'Gasolina de aviación', 'Gasolina regular', 'Gasolina superior',
       'Grasas lubricantes', 'Kerosina', 'Mezclas\noleosas', 'Naftas',
       'PetCoke', 'Petróleo crudo', 'Solventes', 'Butano',
       'Diesel alto azufre', 'Orimulsión', 'Total'],
      dtype='object')


In [171]:
target_columns = ["Fecha","Gas licuado de petróleo", "Gasolina regular", "Gasolina superior", "Diesel alto azufre", "Diesel bajo azufre"]
df_consumo_raw_2024 = df_consumo_raw_2024[target_columns]
print("Obtención de columnas de interés.")
print(f"Consumo 2024: {df_consumo_raw_2024.shape[0]} rows, {df_consumo_raw_2024.shape[1]} columns")

Obtención de columnas de interés.
Consumo 2024: 300 rows, 6 columns


In [172]:
df_consumo_raw_2024.head()

,Fecha,Gas licuado de petróleo,Gasolina regular,Gasolina superior,Diesel alto azufre,Diesel bajo azufre
0,2000-01-01 00:00:00,194410.476190,202645.20,308156.82,634667.06,0.0
1,2000-02-01 00:00:00,174710.552381,205530.96,307766.31,642380.66,0.0
2,2000-03-01 00:00:00,189234.066667,229499.56,331910.29,699807.25,0.0
3,2000-04-01 00:00:00,174330.607143,210680.40,315648.08,586803.98,0.0
4,2000-05-01 00:00:00,191745.147619,208164.34,319667.97,656948.20,0.0


In [173]:
df_consumo_raw_2024['Diesel bajo azufre'] = df_consumo_raw_2024['Diesel alto azufre'] + df_consumo_raw_2024['Diesel bajo azufre']
df_consumo_raw_2024 = df_consumo_raw_2024.drop(['Diesel alto azufre'], axis=1)
df_consumo_raw_2024.head()

,Fecha,Gas licuado de petróleo,Gasolina regular,Gasolina superior,Diesel bajo azufre
0,2000-01-01 00:00:00,194410.476190,202645.20,308156.82,634667.06
1,2000-02-01 00:00:00,174710.552381,205530.96,307766.31,642380.66
2,2000-03-01 00:00:00,189234.066667,229499.56,331910.29,699807.25
3,2000-04-01 00:00:00,174330.607143,210680.40,315648.08,586803.98
4,2000-05-01 00:00:00,191745.147619,208164.34,319667.97,656948.20


In [174]:
print("Ajuste de columnas de consumo 2024 completado.")

Ajuste de columnas de consumo 2024 completado.


In [175]:
df_consumo_raw_2025 = pd.read_excel(CONSUMO_2025_XLSX, skiprows=6)
df_consumo_raw_2025.head()
print(f"Consumo 2025: {df_consumo_raw_2025.shape[0]} rows, {df_consumo_raw_2025.shape[1]} columns")

Consumo 2025: 8 rows, 19 columns


In [176]:
df_consumo_raw_2025.tail()

,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,Gasolina regular,Gasolina superior,Grasas lubricantes,Mezclas\noleosas,Naftas,PetCoke,Petróleo crudo,Solventes,Total
3,2025-04-01 00:00:00,29802.90,5792.94,138234.27,NaN,79483.45,1384488.46,41289.93,581405.92,944.92,878741.74,746081.22,209.25,8090.31,37.24,509893.47,3560.0,11923.71,4419979.73
4,2025-05-01 00:00:00,32191.74,12300.84,160453.73,NaN,81903.71,1416602.71,5696.57,594599.75,NaN,893158.40,761100.59,618.95,8021.11,0.54,537273.51,2163.0,11664.74,4517749.89
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fuente: ventas mensuales obtenidas de informes...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Nota: Información sujeta a actualización por r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
df_consumo_raw_2025 = df_consumo_raw_2025.iloc[:-3]
df_consumo_raw_2025.tail()

,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,Gasolina regular,Gasolina superior,Grasas lubricantes,Mezclas\noleosas,Naftas,PetCoke,Petróleo crudo,Solventes,Total
0,2025-01-01 00:00:00,32041.96,168.80,101775.82,NaN,92410.45,1399798.38,4727.57,581794.25,NaN,828269.64,702319.64,1262.97,5791.72,37.92,399627.73,3522.0,13514.52,4167063.37
1,2025-02-01 00:00:00,28268.31,4023.03,101948.38,388.64,74719.98,1343721.50,83598.78,586541.02,165.78,781113.92,651035.74,341.41,3889.33,46.82,463740.46,2857.0,4392.28,4130792.38
2,2025-03-01 00:00:00,26721.06,8101.52,124247.01,425.03,85437.34,1456533.00,7719.40,589796.81,1495.19,895068.46,763939.48,8375.63,6318.02,0.67,517139.87,2022.0,14447.11,4507787.60
3,2025-04-01 00:00:00,29802.90,5792.94,138234.27,NaN,79483.45,1384488.46,41289.93,581405.92,944.92,878741.74,746081.22,209.25,8090.31,37.24,509893.47,3560.0,11923.71,4419979.73
4,2025-05-01 00:00:00,32191.74,12300.84,160453.73,NaN,81903.71,1416602.71,5696.57,594599.75,NaN,893158.40,761100.59,618.95,8021.11,0.54,537273.51,2163.0,11664.74,4517749.89


In [178]:
print("Eliminación de filas innecesarias completada.")
print(f"Consumo 2024: {df_consumo_raw_2025.shape[0]} rows, {df_consumo_raw_2025.shape[1]} columns")
print(df_consumo_raw_2025.columns)

Eliminación de filas innecesarias completada.
Consumo 2024: 5 rows, 19 columns
Index(['Fecha', 'Aceites lubricantes', 'Asfalto', 'Bunker', 'Ceras',
       'Combustible turbo jet', 'Diesel bajo azufre',
       'Diesel ultra bajo azufre', 'Gas licuado de petróleo',
       'Gasolina de aviación', 'Gasolina regular', 'Gasolina superior',
       'Grasas lubricantes', 'Mezclas\noleosas', 'Naftas', 'PetCoke',
       'Petróleo crudo', 'Solventes', 'Total'],
      dtype='object')


In [179]:
target_columns = ["Fecha","Gas licuado de petróleo", "Gasolina regular", "Gasolina superior", "Diesel bajo azufre"]
df_consumo_raw_2025 = df_consumo_raw_2025[target_columns]
print("Obtención de columnas de interés.")
print(f"Consumo 2025: {df_consumo_raw_2025.shape[0]} rows, {df_consumo_raw_2025.shape[1]} columns")

Obtención de columnas de interés.
Consumo 2025: 5 rows, 5 columns


In [180]:
df_consumo = pd.concat([df_consumo_raw_2024, df_consumo_raw_2025], ignore_index=True)
df_consumo.head()

,Fecha,Gas licuado de petróleo,Gasolina regular,Gasolina superior,Diesel bajo azufre
0,2000-01-01 00:00:00,194410.476190,202645.20,308156.82,634667.06
1,2000-02-01 00:00:00,174710.552381,205530.96,307766.31,642380.66
2,2000-03-01 00:00:00,189234.066667,229499.56,331910.29,699807.25
3,2000-04-01 00:00:00,174330.607143,210680.40,315648.08,586803.98
4,2000-05-01 00:00:00,191745.147619,208164.34,319667.97,656948.20


In [182]:
print("Union de datos de consumo 2024 y 2025 completada.")
df_consumo.to_csv(CONSUMO_CSV, index=False)

Union de datos de consumo 2024 y 2025 completada.


In [116]:
# df_consumo_raw_2024       # DataFrame original de consumo 2024
# df_consumo_raw_2025       # DataFrame original de consumo 2025
# df_consumo                # Unión y limpieza final de consumo mensual

# df_import_raw_2024        # Importaciones 2024
# df_import_raw_2025        # Importaciones 2025
# df_import                 # Unión y limpieza final de importación mensual
# df_import['diesel_total'] # Diésel combinado (alto + bajo azufre)

# df_precios_raw            # Precios diarios
# df_precios                # Promedio mensual gasolina regular


# serie_precio_gasolina_regular      # Series de tiempo de precios mensual (Ciudad Capital)
# serie_import_diesel_total          # Serie mensual de importación total de diésel
# serie_consumo_gas_licuado          # Serie mensual de consumo de gas propano


### Carga y conversión de archivos Excel a CSV

In [ ]:
def loadConsumptionData(path, saveCsv=False, outputPath=None):
    """
    Carga los datos de consumo de hidrocarburos desde un archivo Excel.
    Unifica el diesel bajo y alto azufre en una sola columna.
    """
    df = pd.read_excel(path, skiprows=6)

    # Limpiar nombres de columnas
    df.columns = df.columns.str.strip()

    # Renombrar columnas clave para facilitar análisis
    df = df.rename(columns={
        "Fecha": "fecha",
        "Gasolina regular": "regular",
        "Gasolina superior": "super",
        "Gas licuado de petróleo": "glp",
        "Diesel bajo azufre": "diesel_ls",
        "Diesel alto azufre": "diesel_hs"
    })

    # Crear diesel_total como suma de los dos tipos
    df["diesel_total"] = df.get("diesel_ls", 0) + df.get("diesel_hs", 0)

    # Quedarnos solo con lo necesario
    df = df[["fecha", "regular", "super", "glp", "diesel_total"]]

    # No convertir fechas aún (lo dejamos para limpieza global si es necesario)

    if saveCsv and outputPath:
        df.to_csv(outputPath, index=False)

    return df


In [126]:
CONSUMO_2024_XLSX = "./data-input/CONSUMO-HIDROCARBUROS-2024-12.xlsx"
df_consumo = loadConsumptionData(CONSUMO_2024_XLSX, saveCsv=True, outputPath="./data-output/consumo_2024.csv")


TypeError: loadConsumptionData() got an unexpected keyword argument 'saveCsv'

In [117]:
def sanitizeDateColumn(df, dateColumn="Fecha", convert=False, dropInvalid=False):
    """
    Si convert=True, intenta convertir a datetime.
    Si dropInvalid=True, elimina las fechas inválidas (NaT).
    """
    df = df.copy()
    
    if convert:
        df[dateColumn] = pd.to_datetime(df[dateColumn], errors="coerce")

        if dropInvalid:
            df = df.dropna(subset=[dateColumn])
    
    return df

In [118]:
def loadAndProcessConsumption(saveCsv=True):
    df_2024 = pd.read_excel(CONSUMO_2024_XLSX, skiprows=6)
    df_2025 = pd.read_excel(CONSUMO_2025_XLSX, skiprows=6)

    df_2024.columns = df_2024.columns.str.strip()
    df_2025.columns = df_2025.columns.str.strip()

    rename_map = {
        "Gasolina regular": "regular",
        "Gasolina superior": "super",
        "Diesel bajo azufre": "dieselLS",
        "Diesel alto azufre": "dieselHS",
        "Gas licuado de petróleo": "glp"
    }

    df_2024 = df_2024.rename(columns=rename_map)
    df_2025 = df_2025.rename(columns=rename_map)

    df_2024["diesel_total"] = df_2024.get("dieselHS", 0)
    df_2025["diesel_total"] = df_2025.get("dieselLS", 0)

    df = pd.concat([df_2024, df_2025], ignore_index=True)
    df = df[["Fecha", "regular", "super", "glp", "diesel_total"]]

    # Forzar conversión a datetime y eliminar errores
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df.dropna(subset=["Fecha"])

    df = df.sort_values("Fecha")

    if saveCsv:
        df.to_csv(IMPORT_CSV, index=False)

    return df


In [119]:
def loadAndProcessImportation(saveCsv=True):
    """Carga y unifica los archivos de importación. Crea diesel_total sumando HS y LS según año."""
    df_2024 = pd.read_excel(IMPORT_2024_XLSX, skiprows=6)
    df_2025 = pd.read_excel(IMPORT_2025_XLSX, skiprows=6)

    # Limpia nombres de columnas
    df_2024.columns = df_2024.columns.str.strip()
    df_2025.columns = df_2025.columns.str.strip()

    rename_map_2024 = {
        "Gasolina regular": "regular",
        "Gasolina superior": "super",
        "Diesel bajo azufre": "dieselLS",
        "Diesel alto azufre": "dieselHS",
        "Gas licuado de petróleo": "glp"
    }

    rename_map_2025 = {
        "Gasolina regular": "regular",
        "Gasolina superior": "super",
        "Diesel bajo azufre": "dieselLS",
        "Gas Licuado de Petróleo": "glp"
    }

    df_2024 = df_2024.rename(columns=rename_map_2024)
    df_2025 = df_2025.rename(columns=rename_map_2025)

    # Construir diesel_total por año
    df_2024["diesel_total"] = df_2024.get("dieselHS", 0)
    df_2025["diesel_total"] = df_2025.get("dieselLS", 0)

    df = pd.concat([df_2024, df_2025], ignore_index=True)
    df = df[["Fecha", "regular", "super", "glp", "diesel_total"]]

    # Sanitiza fechas
    df = sanitizeDateColumn(df, "Fecha", convert=True, dropInvalid=True)
    df = df.sort_values("Fecha")

    if saveCsv:
        df.to_csv(IMPORT_CSV, index=False)

    return df

In [120]:
def loadAndProcessPrices(saveCsv=True):
    """Carga precios diarios, renombra columnas clave, limpia fechas y genera promedios mensuales."""
    df = pd.read_excel(PRECIOS_2025_XLSX, skiprows=7)  # Omite encabezado doble

    # Limpieza de nombres de columnas
    df.columns = df.columns.str.strip()

    # Reasigna nombres de columnas manualmente (posición basada en estructura conocida)
    df = df.rename(columns={
        df.columns[0]: "Fecha",
        df.columns[2]: "super",
        df.columns[3]: "regular",
        df.columns[4]: "diesel",
        df.columns[6]: "glp"
    })

    # Filtramos solo columnas relevantes
    df = df[["Fecha", "regular", "super", "diesel", "glp"]]

    # Ahora sí: convertir fechas y eliminar inválidas
    df = sanitizeDateColumn(df, "Fecha", convert=True, dropInvalid=True)

    # Ordenar cronológicamente
    df = df.sort_values("Fecha")

    # Agregación mensual por promedio
    df_mensual = df.set_index("Fecha").resample("M").mean().reset_index()

    if saveCsv:
        df_mensual.to_csv(PRECIOS_CSV, index=False)

    return df_mensual

In [121]:
df_consumo = loadAndProcessConsumption(saveCsv=SAVE_CONSUMO_CSV)
df_importacion = loadAndProcessImportation(saveCsv=SAVE_IMPORT_CSV)
df_precios = loadAndProcessPrices(saveCsv=SAVE_PRECIOS_CSV)


C:\Users\josue\AppData\Local\Temp\ipykernel_25704\3229522666.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_mensual = df.set_index("Fecha").resample("M").mean().reset_index()


In [122]:
# # Re-guardar por si SAVE_* son False
# df_consumo.to_csv(CONSUMO_CSV, index=False)
# df_importacion.to_csv(IMPORT_CSV, index=False)
# df_precios.to_csv(PRECIOS_CSV, index=False)

# # Verificación de forma (shape)
# print("✔ Data saved and loaded:")
# print(f"Consumo: {df_consumo.shape}")
# print(f"Importación: {df_importacion.shape}")
# print(f"Precios: {df_precios.shape}")
